# Librairies

In [ ]:
import pickle
import numpy as np
import os
import cv2
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode


# Création du dataset

In [ ]:
# Création du répertoire data si inexistant
if not os.path.exists("data"):
    os.mkdir("data")

# Nombre d'images qu'on collecte pour la reconnaissance faciale
nb_images_collectees = 10

# Initialisation des listes pour stocker les données des visage
donnees_visage = []

# Fonction pour capturer une image à partir de la webcam de l'utilisateur
def take_photo(filename='photo.jpg', quality=0.8):

    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');// bouton pour capture d'image
        capture.textContent = 'Capture';
        div.appendChild(capture);

        // permet l'affichage du flux vidéo de notre caméra en temps réel
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // l'exécution du script s'interromp jusqu'à ce qu'un clic pour la capture soit effectué
        await new Promise((resolve) => capture.onclick = resolve);

        //capture de l'image, arrêt de la caméra, mise en format jpeg de l'image
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Chargement d'un modèle pré-entraîné de détection de visages fourni par OpenCV
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Demande à l'utilisateur de saisir le nom de la personne qui sera prise en photo, ce même nom sra associéà toutes les images capturées
nom = input("Entrez votre nom: ")  # Demande du nom de l'utilisateur

noms = []  # Initialisation d'un liste vide pour stocker les noms correspondant aux visages capturés
for i in range(nb_images_collectees):
    filename = take_photo()  # Capture d'une nouvelle photo
    image = cv2.imread(filename)
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5) #Utilisation du modèle de détection de visages pour trouver les visages dans l'image
    visage_trouve = False

    # Traitement de chaque visage détecté dans l'image
    for (x, y, w, h) in coordonnees_visage:
        # Pour chaque visage détecté, extrait, redimensionne, et aplati le visage avant de l'ajouter à la liste des données de visage
        visage = image[y:y+h, x:x+w]
        visage_redimensionne = cv2.resize(visage, (50, 50))
        # Aplatissement de l'image redimensionnée pour la préparation avant l'entraînement du modèle.
        donnees_visage.append(visage_redimensionne.flatten())
        visage_trouve = True
        break

    if visage_trouve:
        noms.append(nom)  # Association du nom de l'utilisateur au visage capturé

Entrez votre nom: khaoula


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Algorithme et exécution

In [ ]:
# Conversion des listes en tableaux numpy
donnees_visage = np.array(donnees_visage)

In [ ]:
# Enregistrement des noms et des visages
noms = [nom] * nb_images_collectees
with open("data/noms.pkl", 'wb') as file:
    pickle.dump(noms, file)

with open("data/visages.pkl", 'wb') as file:
    pickle.dump(donnees_visage, file)

In [ ]:
# Entraînement de l'arbre de décision
X_train, X_test, y_train, y_test = train_test_split(donnees_visage, noms, test_size=0.2, random_state=42)
arbre_decision = DecisionTreeClassifier(max_depth=10)
arbre_decision.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10)

In [ ]:
# Capture d'un nouveau visage pour la prédiction
filename = take_photo()  # Capture d'une nouvelle photo
image = cv2.imread(filename) # lexture de l'img capturée
gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #image mise en niveaux de gris

#Utilisation du modèle de détection de visages pour trouver les visages dans l'image
coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5) # Détecte les visages dans l'image

visage_predire = None

for (x, y, w, h) in coordonnees_visage:
  # Extraction du visage détecté à partir de l'image originale
    visage = image[y:y+h, x:x+w]

    visage_redimensionne = cv2.resize(visage, (50, 50)) # Redimensionnement d visage à une taille standard (en pixels)
    visage_predire = visage_redimensionne.flatten().reshape(1, -1)  # Aplatisssement du visage pour la prédiction
    break

# Prédiction du nouveau visage
if visage_predire is not None:
    prediction = arbre_decision.predict(visage_predire) # Utilisation du modèle de prédiction pour identifier le visage
    print(f"Visage reconnu comme : {prediction[0]}") # Affichage du résultat de la prédiction
else:
    print("Aucun visage détecté pour la prédiction.") # Message d'erreur qui affiche si aucun visage n'est détecté

<IPython.core.display.Javascript object>

Visage reconnu comme : khaoula
